In [1]:
# ! pip install transformers==4.20.0
# ! pip install datasets>=2.2.0
# ! pip install jsonlines
# ! pip install numpy
# ! pip install requests>=2.26.0
# ! pip install scikit_learn>=1.0.2
# ! pip install scipy
# ! pip install sentence_transformers>=2.2.0
# ! pip install torch
# ! pip install tqdm
# ! pip install rich
# ! pip install Cython

In [2]:
# ! pip install PyPDF2 langchain InstructorEmbedding torch

from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


from langchain.chains.question_answering import load_qa_chain

# from htmlTemplates import css, bot_template, user_template
from langchain.llms import HuggingFaceHub

import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

### Read pdf File & split into chunks

In [3]:
filename = "/Users/krishnasai/Downloads/HP1 - Harry Potter and the Sorcerer's Stone.pdf"

doc = PdfReader(filename)
text = ""
for page in doc.pages:
    text += page.extract_text()

len(text)

462945

In [2]:
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=400,
        chunk_overlap=80,
        length_function=len
)
chunks = text_splitter.split_text(text) 
len(chunks)

NameError: name 'CharacterTextSplitter' is not defined

### Download Embeddings

In [5]:
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", )

/Users/krishnasai/miniconda3/envs/llm/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


### Remove old inexes & build a new one

In [6]:
from langchain.vectorstores import Pinecone
import pinecone

pinecone.init(
    api_key=os.environ.get("PINECONE_API_KEY"),
    environment=os.environ.get("PINECONE_ENV"),
)
print(pinecone.info.version())

# # Delete all indexes
# indexes = pinecone.list_indexes()
# for i in indexes: 
#     pinecone.delete_index(i)

# Create a new index
index_name = 'harry-potter'
# index = pinecone.create_index(index_name, dimension=768,)
# pinecone.list_indexes()

VersionResponse(server='2.0.11', client='2.2.2')


In [7]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.01549,
 'namespaces': {'': {'vector_count': 1549}},
 'total_vector_count': 1549}

### Upload embeddings of chunks into pinecone database 

In [8]:
# vector_store = Pinecone.from_texts(
#     texts= chunks,
#     embedding= embeddings,
#     index_name= index_name
# )
# index.describe_index_stats()

In [9]:
# index.describe_index_stats()
vector_store = Pinecone.from_existing_index(
                                index_name, 
                                embeddings)

### Retrieve relevant chunks relaed to queries

In [10]:
# relevant_chunks = vector_store.similarity_search("Where does harry study?")

# for chunk in relevant_chunks:
#     print(chunk)
#     print("-----------")

### LLM Retriever Chain 
#### to fetch the relevant chunks & answer the question

In [11]:
llm = HuggingFaceHub(
#     repo_id="google/flan-t5-xxl"
    repo_id = 'meta-llama/Llama-2-70b-chat'
    , model_kwargs={"temperature":0.01}
)

memory = ConversationBufferMemory(
    memory_key='chat_history'
    , return_messages=True
)


chat_chain = ConversationalRetrievalChain.from_llm(
    llm=llm
    , retriever=vector_store.as_retriever()
    , memory=memory
)


In [12]:
query = input()

# docs = vector_store.similarity_search(query)

r = chat_chain({"question": query})
r['answer']

who is the principal of Hogwarts?


ValueError: Error raised by inference API: meta-llama/Llama-2-70b-chat does not appear to have a file named config.json. Checkout 'https://huggingface.co/meta-llama/Llama-2-70b-chat/cfedb123eabd84594950cb9c9b429d2f0bebd627' for available files.